In [326]:
# Chase Abram
# using Pkg
# Pkg.add("DataFrames")
# using CSV
# using DataFrames
# # df = DataFrame()
# df = CSV.read("psetOne.csv", DataFrame)

using LinearAlgebra
using Optim

using Random
rng = MersenneTwister(12345)

MersenneTwister(UInt32[0x00003039], Random.DSFMT.DSFMT_state(Int32[-870096391, 1072918504, -1812426662, 1073255081, -733866021, 1073404543, 807620846, 1073368448, 1919433844, 1072852359  …  -362113007, 1073100625, -166402106, 1073460158, -1907020342, 721295190, -750225566, -1300227565, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000

In [161]:
names(df)
df17 = df[df."Market" .== 17, :]
# df17."shares"
# X = convert(Matrix, df17[:, filter(x -> x in ["Constant", "EngineSize", "SportsBike", "Brand2", "Brand3"], names(df17))])
Br = convert(Matrix, df17[:, filter(x -> x in ["Brand2", "Brand3"], names(df17))])
Br = Bool.(Br)

7×2 BitArray{2}:
 0  0
 0  0
 1  0
 1  0
 0  1
 0  1
 0  1

In [482]:
function u_bar_jt(delta, sigma, X, zeta)
    return delta .+ X*sigma*zeta
end

function Prij(u_bar)
    return exp.(u_bar)./sum(exp.(u_bar))
end


function s_logit(p, alpha, beta, X, xi)
    # Computer nmuerators for each s_j
    nums = exp.(X*beta .- alpha.*p[2:end] .+ xi)
    
    # return s_j, starting with s_0
    return [1/(1 + sum(nums)); nums./(1 + sum(nums))]
end

# Negative jac
function s_logit_jac(p, alpha, beta, X, xi)
    # Get shares
    sp = s_logit(p, alpha, beta, X, xi)
    
    # Jacobian is the outer product times alpha in this case...
    jac = alpha .* sp * sp'
    
    # ... except for an extra term on the diag
    for j in 1: length(jac[:,1])
        jac[j,j] -= alpha*sp[j]
    end
    
    return jac
end

# ISSUE PROBABLY HERE
function s_logit_hess(p, alpha, beta, X, xi)
    hess = zeros(length(p), length(p), length(p))
    for j in 1:length(p)
        for k in 1:length(p)
            for l in 1:length(p)
                s = s_logit(p, alpha, beta, X, xi)
                sj = s_logit_jac(p, alpha, beta, X, xi)
                if j == k && j == l
                    hess[j,k,l] = sj[l,j]*alpha*(2*s[j] - 1)
                elseif j == k
                    hess[j,k,l] = sj[l,j]*alpha*(2*s[j] - 1)
                elseif j == l
                    hess[j,k,l] = sj[k,j]*alpha*(2*s[j] - 1)
                elseif k == l
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                else
                    hess[j,k,l] = alpha*(sj[l,j]*s[k] + s[j]*sj[l,k])
                end
            end
        end
    end
    
    return hess
end

function Omega_star(br)
    Om = zeros(size(br,1) + 1,size(br,1) + 1)
    for i in 1:size(Om,1) - 1
        for j in 1:size(Om,2) - 1
            if br[i,1] && br[j,1]
                Om[i+1,j+1] = 1
            elseif br[i,2] && br[j,2]
                Om[i+1,j+1] = 1
            elseif (!br[i,1]&& !br[j,1]) && (!br[i,2]&& !br[j,2])
                Om[i+1,j+1] = 1
            end
        end
    end
    Om[1,1] = 1
    return Om
end

function Omega(p, alpha, beta, X, xi, br)
    return Omega_star(br) .* s_logit_jac(p, alpha, beta, X, xi)
end

function fp_logit(p, alpha, beta, X, xi, br)
    return s_logit(p, alpha, beta, X, xi) + Omega(p, alpha, beta, X, xi, br)*p
end

function jac_om_p(p, alpha, beta, X, xi, br)
    jacomp = zeros(length(p), length(p))
    hess = s_logit_hess(p, alpha, beta, X, xi)
    for i in 1:length(p)
        for j in 1:length(p)
            jacomp[i,j] = ((hess[i,:,j].* Omega_star(br)[i,j])'* p)[1]
        end
    end
    return jacomp
end

function fp_logit_2(p, alpha, beta, X, xi, br)
    return fp_logit(p, alpha, beta, X, xi, br)' * fp_logit(p, alpha, beta, X, xi, br)
end


function fp_logit_2_jac(storage, p, alpha, beta, X, xi, br)
    sp = s_logit(p, alpha, beta, X, xi)
    Om = Omega_star(br).* s_logit_jac(p, alpha, beta, X, xi)
    s_jac = s_logit_jac(p, alpha, beta, X, xi)
    jacomp = jac_om_p(p, alpha, beta, X, xi, br)
    
    jac = (sp + Om*p)' * (s_jac + jacomp + Om)
    for i in 1:length(p)
        storage[i] = jac[i]
    end
#     storage = (sp + Om*p)' * (s_jac + jacomp + Om)
#     storage = 0
#     return storage
    
end

function solve_fp_logit(alpha, beta, X, xi, br, tol = 1e-14, maxit = 100)
    
    f(x) = fp_logit_2(x, alpha, beta, X, xi, br)[1,1]
    g!(storage, x) = fp_logit_2_jac(storage, x, alpha, beta, X, xi, br)
    
#     opt = optimize(f, g!, p, NelderMead())
    opt = optimize(f, g!, p, LBFGS())
    return opt, opt.minimizer, opt.minimum
#     return opt
end



solve_fp_logit (generic function with 3 methods)

In [483]:
p = ones(size(X,1)+1,1)
p[7] = 1
alpha = 3
beta = [1 1 2 -1 1]'

# seed maintains results across runs
rng = MersenneTwister(123)
xi = randn(rng, size(df17,1))

s_logit(p, alpha, beta, X, xi)
println("sum shares: ", sum(s_logit(p, alpha, beta, X, xi)))

s_logit_jac(p, alpha, beta, X, xi)

# s_logit_hess(p, alpha, beta, X, xi)

# Omega_star(Br)

# Omega(p, alpha, beta, X, xi, Br)

# fp_logit(p, alpha, beta, X, xi, Br)' * fp_logit(p, alpha, beta, X, xi, Br)

# fp_logit_2(p, alpha, beta, X, xi, Br)
fp_logit_2_jac(p, alpha, beta, X, xi, Br)

# jac_om_p(p, alpha, beta, X, xi, Br)

# fp_logit_jac(p, alpha, beta, X, xi, Br)


solve_fp_logit(alpha, beta, X, xi, Br)

# op = optimize(x -> (fp_logit(x, alpha, beta, X, xi, Br)' * fp_logit(x, alpha, beta, X, xi, Br))[1,1], p, LBFGS())

# pmin = op.minimizer
# println("p min: ", pmin)
# println("min: ", op.minimum)
# println("fp_logit: ", fp_logit(pmin, alpha, beta, X, xi, Br))
# df17[3, "Brand2"]

sum shares: 1.0


LoadError: InterruptException:

In [430]:
function sHat(delta, X, sigma, zeta)
    
    
    # Compute u_bar_ijt
    
    # Compute prob.
    
    # Do above I times, then integrate
    
    
    return 0
end



sHat (generic function with 1 method)

In [456]:
function tester()
    f1(x, y, z) = (x[1] - 5 - y)^2 + (x[2] - 3 - z)^2
    
    function g(storage, x, y, z)
        storage[1] = 2*(x[1] - 5 - y)
        storage[2] = 2*(x[2] - 3 - z)
    end
    
    a = 40
    b = 5
    f(x) = f1(x,a,b)
    g!(storage, x) = g(storage, x, a, b)
    
    
    opt = optimize(f, g!, [0.0, 0.0], LBFGS())
    return opt, opt.minimizer, opt.minimum
end

tester()

( * Status: success

 * Candidate solution
    Final objective value:     0.000000e+00

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 4.50e+01 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.00e+00 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.09e+03 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = Inf ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3
, [45.0, 8.0], 0.0)